# ADPA Integrations

This notebook demonstrates how to integrate ADPA with popular tools and frameworks:
- Streamlit for web applications
- FastAPI for REST APIs
- Pandas for data analysis
- SQLAlchemy for ORM
- Jupyter widgets for interactive dashboards

## Setup

First, let's import all necessary components.

In [ ]:
import streamlit as st
from fastapi import FastAPI, HTTPException
import pandas as pd
from sqlalchemy import create_engine
import ipywidgets as widgets
from IPython.display import display

from adpa import Text2SQL, Database
from adpa.config import Config
from adpa.integrations import (
    StreamlitIntegration,
    FastAPIIntegration,
    PandasIntegration,
    SQLAlchemyIntegration
)

# Initialize components
config = Config()
config.load_env()

db = Database(config)
text2sql = Text2SQL(config)

## Streamlit Integration

Create an interactive web application using Streamlit.

In [ ]:
# Initialize Streamlit integration
streamlit_app = StreamlitIntegration(text2sql, db)

def create_streamlit_app():
    st.title("ADPA Text2SQL Explorer")
    
    # Add query input
    query = st.text_area("Enter your query:")
    if st.button("Execute"):
        with st.spinner("Processing query..."):
            # Execute query
            result = streamlit_app.execute_query(query)
            
            # Display results
            st.write("Generated SQL:")
            st.code(result.sql)
            
            st.write("Results:")
            st.dataframe(result.data)
            
            # Show performance metrics
            st.write("Performance Metrics:")
            st.json(result.metrics)

# Note: This code would be in a separate .py file
# streamlit run app.py

## FastAPI Integration

Create a REST API using FastAPI.

In [ ]:
# Initialize FastAPI integration
fastapi_app = FastAPIIntegration(text2sql, db)

def create_fastapi_app():
    app = FastAPI(title="ADPA API")
    
    @app.post("/query")
    async def execute_query(query: str):
        try:
            result = await fastapi_app.execute_query(query)
            return {
                "sql": result.sql,
                "data": result.data,
                "metrics": result.metrics
            }
        except Exception as e:
            raise HTTPException(status_code=400, detail=str(e))
    
    return app

# Note: This code would be in a separate .py file
# uvicorn app:app --reload

## Pandas Integration

Integrate with Pandas for data analysis.

In [ ]:
# Initialize Pandas integration
pandas_integration = PandasIntegration(text2sql, db)

# Example queries
queries = [
    "Show me sales by region",
    "Calculate monthly revenue growth",
    "Find top 10 products by profit margin"
]

# Execute queries and analyze results
for query in queries:
    print(f"\nProcessing query: {query}")
    
    # Get results as DataFrame
    df = pandas_integration.query_to_dataframe(query)
    
    # Perform analysis
    print("\nSummary Statistics:")
    print(df.describe())
    
    print("\nCorrelation Matrix:")
    print(df.corr())
    
    # Create visualizations
    pandas_integration.plot_results(df, query)

## SQLAlchemy Integration

Use SQLAlchemy for ORM capabilities.

In [ ]:
# Initialize SQLAlchemy integration
sqlalchemy_integration = SQLAlchemyIntegration(text2sql, db)

# Define models
Base = sqlalchemy_integration.get_declarative_base()

class Employee(Base):
    __tablename__ = "employees"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    department = Column(String)
    salary = Column(Float)

# Create session and execute queries
with sqlalchemy_integration.session() as session:
    # Natural language query using ORM
    query = "Find employees in Engineering with salary above 70000"
    results = sqlalchemy_integration.execute_orm_query(query, Employee)
    
    for employee in results:
        print(f"{employee.name}: {employee.salary}")

## Interactive Dashboard

Create an interactive dashboard using Jupyter widgets.

In [ ]:
# Create widgets
query_input = widgets.Textarea(
    value='',
    placeholder='Enter your query',
    description='Query:',
    disabled=False
)

execute_button = widgets.Button(
    description='Execute',
    disabled=False,
    button_style='success'
)

output = widgets.Output()

# Define callback
def on_execute_button_clicked(b):
    with output:
        output.clear_output()
        query = query_input.value
        
        try:
            # Execute query
            result = text2sql.execute(query)
            
            # Convert to DataFrame
            df = pd.DataFrame(result)
            
            # Display results
            print("Results:")
            display(df)
            
            # Create visualization
            if len(df) > 0:
                df.plot(kind='bar')
        except Exception as e:
            print(f"Error: {e}")

execute_button.on_click(on_execute_button_clicked)

# Display dashboard
display(widgets.VBox([query_input, execute_button, output]))

## Cleanup

Clean up resources.

In [ ]:
# Close database connection
db.close()

# Cleanup integrations
streamlit_app.cleanup()
fastapi_app.cleanup()
pandas_integration.cleanup()
sqlalchemy_integration.cleanup()